# Exercice Final OpenAI

Faire soi-même un assistant OpenAI
Un assistant qui est à la fois capable :
- D’aller chercher les top 3 news à propos d’un sujet, à partir d’une date max ou pas, sur [NewsAPI](https://newsapi.org/), puis les résumer et les renvoyer sous forme de JSON avec la date, titre et résumé de l’article.

- D’aller chercher des informations sur la [certification Dev IA 2023](https://github.com/louiskuhn/IA-P3-Euskadi/blob/main/Ressources/GenAI/OpenAI_Assistants/reglement_specifique_full_dev_ia_2023.pdf) en réutilisant le vector_store de l’exercice.

L’assistant doit être robuste aux injections de prompt et ne doit rien faire d’autre que les tâches ci-dessus. Il doit répondre en français et présenter le moins possible d’hallucinations.

Puis faire une interface pour le tout, avec [panel](https://panel.holoviz.org/) ou [gradio](https://www.gradio.app/guides/quickstart) par exemple.

In [1]:
import os
import json
from datetime import date
import time

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())


In [2]:
from openai import AzureOpenAI, OpenAI

openai_api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI(
    api_key=openai_api_key    
)

## Création des instructions de l'assistant 

Très important de bien définir l'entrée et la sortie du modèle, ainsi que tous les différents cas de figures

Il n'est pas parfait, vous pouvez essayer de jouer avec

In [3]:
# Version avec uniquement les news

assistant_instructions = """
Your task is to get 3 news articles based on the topic given by the user.\
The user may or may not provide a starting date for the articles.\
Use the provided functions to answer the questions.\
The 3 news articles must be returned in json format with the following schema : 

{
  "status": "failed, or succeeded"
  "answer": "empty string, or the error"
  "articles":
  [
    {
      "article_title": "Title of the article in one sentence",
      "article_source": "Name of the source of the article",
      "article_description": "Description of the article",
      "article_link": "url link to the article",
      "article_date": "date the article was published at"
    }
  ]
}

If the user gives no topic, answer the error "Je ne suis pas sûr d'avoir compris, donnez-moi un sujet"\
, unless the getNewsArticles function returned a non-empty list of articles
If there is no news about the topic, that is if the getNewsArticles function is called and it returns an empty list of articles\
, answer the error "Je n'ai pas trouvé d'articles à propos de ce sujet" 
If the user asks for something else than news, answer the error "Je suis désolé, je ne peux que fournir des news"
For any of the three failed cases above, return nothing in the article and a failed status

"""

function_description = """
Given keywords or a phrase to query, this function gets the 3 most \
relevant news articles. If a starting date is given, it will get \
articles published between the starting date and the present date. \
Expects a list of articles in json format as a response.
"""

In [4]:
# Version avec news + questions certif

assistant_instructions_full = """
You have 2 possible use cases : Either you answer questions about the certification delivered by Simplon,
or you get 3 news articles based on the topic given by the user.\
The user may or may not provide a starting date for the articles.\
Use the provided functions and file search to answer the questions.\

First, you will determine if the user is asking for news articles about a topic, that we will call use case 1, \
or if he's asking for information about the certification delivered by Simplon, that we will call use case 2.
Once this is done:
For use case 1:
The 3 news articles must be returned in json format with the following schema : 

{
  "status": "failed, or succeeded",
  "answer": "empty string, or the error",
  "articles":
  [
    {
      "article_title": "Title of the article in one sentence",
      "article_source": "Name of the source of the article",
      "article_description": "Description of the article",
      "article_link": "url link to the article",
      "article_date": "date the article was published at"
    }
  ]
}

If the user gives no topic, answer the error "Je ne suis pas sûr d'avoir compris, donnez-moi un sujet"\
, unless the getNewsArticles function returned a non-empty list of articles
If there is no news about the topic, that is if the getNewsArticles function is called and it returns an empty list of articles\
, answer the error "Je n'ai pas trouvé d'articles à propos de ce sujet" 
If the user asks for something else than news, answer the error "Je suis désolé, je ne peux que fournir des news"
For any of the three failed cases above, return nothing in the article and a failed status

For use case 2 :
Try to find relevant sources in your resources and only answer the question based on those sources.
The response must be returned ONLY in json format with the following schema:
{
  "status": "failed, or succeeded",
  "answer": "the answer of the model in its entirety with citation numbers between brackets, or the error",
  "citations": {
    citation number: "citation file name"
  }
  
}
If relevant sources to answer to the question asked could not be found in  your resources, or if you are not certain, answer the error \
"Désolé, nous n'avons pas pu trouvé la réponse à votre question dans nos données"
For the error case described above, return a failed status and an empty citations list

The response must absolutely and only be returned in the json format specified for both use cases.
"""

## Création de l'assistant et de sa fonction

Important de bien définir la description de la fonction et ses paramètres

In [5]:
# Version avec uniquement les news


# assistant = client.beta.assistants.create(
#   instructions=assistant_instructions,
#   name="news_assistant",
#   model='gpt-3.5-turbo', #Replace with model deployment name
#   tools=[{
#       "type": "function",
#     "function": {
#       "name": "getNewsArticles",
#       "description": function_description,
#       "parameters": {
#         "type": "object",
#         "properties": {
#           "query": {"type": "string", "description": "Keywords or a phrase to query articles."},
#           "limit_date": {"type": "string", "description": "starting date for articles in ISO 8601 format."}
#         },
#         "required": ["query"]
#       }
#     }
#   }],
#   response_format={ "type": "json_object" }
# )
# print(assistant) # asst_Wfgt7Q88W9FOFNWqmVd69ZMB

In [6]:
# Version avec news + questions certif


# assistant = client.beta.assistants.create(
#   instructions=assistant_instructions,
#   name="news_and_certification_assistant",
#   model='gpt-3.5-turbo', #Replace with model deployment name
#   tools=[{
#       "type": "function",
#     "function": {
#       "name": "getNewsArticles",
#       "description": function_description,
#       "parameters": {
#         "type": "object",
#         "properties": {
#           "query": {"type": "string", "description": "Keywords or a phrase to query articles."},
#           "limit_date": {"type": "string", "description": "starting date for articles in ISO 8601 format."}
#         },
#         "required": ["query"]
#       }
#     }
#   },
#   {"type": "file_search"}],
#   tool_resources={
#     "file_search": {
#       "vector_store_ids": ["vs_vUsPdfeq1ymnt0mC3BuU39Bq"]
#     }
#   }
# )
# print(assistant) # asst_jbpMGP39cEMyhm6eD8JKO02r

## Fonction pour créer un thread et lui ajouter le prompt initial

In [7]:
def create_thread_and_message(message_content):
  thread = client.beta.threads.create()
  print(thread) # thread_xw9ufX67vrbyZljmRnfxuc3f

  # Add a user question to the thread
  message = client.beta.threads.messages.create(
      thread_id=thread.id,
      role="user",
      content=message_content
  )
  return thread
# thread = create_thread_and_message("give me news about the olympic games in France from the last 3 days")

## Fonction d'appel d'API

In [8]:
import requests
from datetime import datetime, timedelta
import urllib.parse

news_api_key = os.getenv("NEWSAPI_API_KEY")

def getNewsArticles(**args):
  try:
    query = args.get('query')
    limit_date = args.get('limit_date')
    print(query)
    print(limit_date)      
    
    url = 'https://newsapi.org/v2/everything'
    params = {
        'q': urllib.parse.quote(query),
        'apiKey': news_api_key,
        'sortBy': 'popularity',
        'pageSize': 3,
        'language': 'fr'
    }
    if limit_date:
      today_date = date.today().strftime(format="%Y-%m-%d")

      if limit_date == today_date: # because the API doesn't work correctly with today's date as limit date
        limit_date = datetime.strptime(limit_date, "%Y-%m-%d") - timedelta(days=1)
        limit_date = limit_date.strftime(format="%Y-%m-%d")

      params['from'] = limit_date
        

    headers = {
        'accept': 'application/json'
    }
    
    response = requests.get(url, params=params, headers=headers)
    print(response)
    if response.status_code == 200:
      data = response.json()
      article_data = data.get('articles')
      print(article_data)
      
      return json.dumps(article_data)
    else:
        error_message = f"Failed to retrieve data from API : {response.status_code}"
        error_dict = {"status": error_message}
        return json.dumps(error_dict)
  except Exception as e:
    print(e)
    return '{"status": "failed API Call"}'


In [9]:
# arguments = json.loads('{\"query\":\"elections americaines\",\"limit_date\":\"2024-09-23"}')
# news = getNewsArticles(**arguments)
# print(json.dumps(json.loads(news), indent=2))

## Fonction répondant à tous les tool_calls

In [10]:
def submit_function_outputs(run, thread):
  if run.required_action.submit_tool_outputs.tool_calls:
    tool_calls = run.required_action.submit_tool_outputs.tool_calls
    
    tool_outputs = []
    for i, tool_call in enumerate(tool_calls):
      function_name = tool_call.function.name
      arguments = json.loads(tool_call.function.arguments)
      response = globals()[function_name](**arguments)

      tool_outputs.append({
        "tool_call_id": tool_call.id,
        "output": response # doit contenir la réponse de votre fonction python qui prend en entrée les arguments renvoyés par le run
      })

    run = client.beta.threads.runs.submit_tool_outputs_and_poll(
      thread_id=thread.id,
      run_id=run.id,
      tool_outputs=tool_outputs
    )
  return run

## Fonction regroupant tout, pour permettre de tester un prompt en une seule ligne

In [11]:
from IPython.display import clear_output

def test_assistant(assistant_id, message_content):


  thread = create_thread_and_message(message_content)

  today_date = date.today().strftime(format="%Y-%m-%d")

  assistant = client.beta.assistants.update(
    assistant_id,
    instructions=assistant_instructions_full + f" Today's date in ISO 8601 format is {today_date}."
  )                                         

  run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
  )

  start_time = time.time()

  status = run.status

  while status not in ["completed", "cancelled", "expired", "failed", "incomplete"]:
    
    run = client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id)
    print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
    status = run.status
    print(f'Status: {status}')

    if run.status == "requires_action":
      print('executing tools...')
      run = submit_function_outputs(run, thread)
      status = run.status

    time.sleep(2)

  if status == "completed":
    messages = client.beta.threads.messages.list(
      thread_id=thread.id
    )
    # clear_output(wait=True)
    print(f'Status: {status}')
    print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))

    model_response = messages.data[0].content[0].text.value


    model_response = model_response.strip('` \n')
    if model_response.startswith('json'):
        model_response = model_response[4:]

    citations = []
    if getattr(messages.data[0].content[0].text, 'annotations'):
      annotations = messages.data[0].content[0].text.annotations
      
      for index, annotation in enumerate(annotations):
        model_response = model_response.replace(annotation.text, f"[{index}]")
        if file_citation := getattr(annotation, "file_citation", None):
          cited_file = client.files.retrieve(file_citation.file_id)
          citations.append(f"[{index}] {cited_file.filename}")

    try:
      print(json.dumps(json.loads(model_response), indent=2).encode('utf-8').decode('unicode_escape'))
    except:
      print("couldn't print json response")
      print(model_response)
    
    if citations:
      print(citations)
      


  else:
    if getattr(run, 'last_error'):
      print(f"Last error: {run.last_error}")
    if getattr(run, 'incomplete_details'):
      print(run.incomplete_details)
    print(f"Assistant status: {status}")
  
  
  
      
  return messages
  

In [12]:
import gradio as gr



def run_assistant_chatbot(message, history,
                          request: gr.Request,
                          assistant_id="asst_zgsWyBwKdWenyzpi9hGEl5h6"):
    
  session_thread = create_thread_and_message(message)

  today_date = date.today().strftime(format="%Y-%m-%d")

  assistant = client.beta.assistants.update(
    assistant_id,
    instructions=assistant_instructions_full + f"Today's date in ISO 8601 format is {today_date}."
  )

  start_time = time.time()
  run = client.beta.threads.runs.create_and_poll(
    thread_id=session_thread.id,
    assistant_id=assistant_id,
  )

  print(f'Status: {run.status}')
  print("Elapsed time before function call: {} minutes {} seconds"
        .format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))


  if run.status == "requires_action":
    print('executing tools...')
    run = submit_function_outputs(run, session_thread)
    print("Elapsed time after function call: {} minutes {} seconds"
        .format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
  # for debugging
  else:
    if getattr(run, 'last_error'):
      print(f"Last error: {run.last_error}")
    if getattr(run, 'incomplete_details'):
      print(run.incomplete_details)
    print(f'Status: {run.status}')

  if run.status == "completed":
    messages = client.beta.threads.messages.list(
      thread_id=session_thread.id
    )

    print(f'Status: {run.status}')
    print("Elapsed time total: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))

    model_response = messages.data[0].content[0].text.value

    # Comme on a une réponse en JSON, on peut l'analyser pour modifier la réponse envoyée à l'utilisateur.

    try:
      json_model_response = json.loads(model_response)
      if json_model_response.get('status') == 'failed':
        return json_model_response.get('answer')
      else:
        articles = json.dumps(json_model_response.get('articles'), indent=2).encode('utf-8').decode('unicode_escape')
        for index in range(len(articles)):
          time.sleep(0.01)
          yield articles[0:index+1]
    except Exception as e:
      return "could not read model response"
    

  else:
    if getattr(run, 'last_error'):
      print(f"Last error: {run.last_error}")
    if getattr(run, 'incomplete_details'):
      print(run.incomplete_details)
    return f"Run status: {run.status}"


gr.ChatInterface(
    run_assistant_chatbot,
    chatbot=gr.Chatbot(height=400),
    textbox=gr.Textbox(placeholder="Votre question ici", container=False, scale=7),
    title="Le bot de la certification développeur IA mais aussi des news",
    description="Pose moi des questions sur ta certification ou sur les nouvelles que tu souhaites",
    theme="soft",
    examples=["Quel est le programme de la certification IA 2023?", "Quelles sont les news sur l'IA cette semaine?"],
    cache_examples=False,
    retry_btn=None,
    undo_btn=None,
    clear_btn=None,
).launch(share=True) # share=True

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://aa40ed31093925c65a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Thread(id='thread_vbZJk8Qrvao0kYxVbOY0XeBl', created_at=1727677993, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))
Status: completed
Elapsed time before function call: 0 minutes 11 seconds
Status: completed
Status: completed
Elapsed time total: 0 minutes 11 seconds


Traceback (most recent call last):
  File "c:\Users\luca5\.conda\envs\GENAI\Lib\site-packages\gradio\chat_interface.py", line 652, in _stream_fn
    first_response = await async_iteration(generator)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\luca5\.conda\envs\GENAI\Lib\site-packages\gradio\utils.py", line 663, in async_iteration
    return await iterator.__anext__()
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\luca5\.conda\envs\GENAI\Lib\site-packages\gradio\utils.py", line 656, in __anext__
    return await anyio.to_thread.run_sync(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\luca5\.conda\envs\GENAI\Lib\site-packages\anyio\to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\luca5\.conda\envs\GENAI\Lib\site-packages\anyio\_backends\_asyncio.py", line 2134, in run_sync_in_worker_thread
    return await futu

Thread(id='thread_CstW5lGOS8DPkzZvmC2G1vqQ', created_at=1727678046, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))
Status: completed
Elapsed time before function call: 0 minutes 9 seconds
Status: completed
Status: completed
Elapsed time total: 0 minutes 9 seconds


Traceback (most recent call last):
  File "c:\Users\luca5\.conda\envs\GENAI\Lib\site-packages\gradio\chat_interface.py", line 652, in _stream_fn
    first_response = await async_iteration(generator)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\luca5\.conda\envs\GENAI\Lib\site-packages\gradio\utils.py", line 663, in async_iteration
    return await iterator.__anext__()
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\luca5\.conda\envs\GENAI\Lib\site-packages\gradio\utils.py", line 656, in __anext__
    return await anyio.to_thread.run_sync(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\luca5\.conda\envs\GENAI\Lib\site-packages\anyio\to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\luca5\.conda\envs\GENAI\Lib\site-packages\anyio\_backends\_asyncio.py", line 2134, in run_sync_in_worker_thread
    return await futu

### Essayez vous même

In [35]:
# bout de code pour éviter de créer plein de fois le même vector store
try : 
  vector_store = client.beta.vector_stores.retrieve(
    vector_store_id="vs_lrYKplXCKWpkkgtXUXzeanNt"
  )
  print(vector_store)
  print("vector store already exists")

except Exception as e:
  print("vector store not found")

  # Create a vector store caled "Certif 5 Pages IA 2023"
  vector_store = client.beta.vector_stores.create(name="Certif 5 Pages IA 2023")
  
  # Ready the files for upload to OpenAI
  file_paths = ["reglement_specifique_5_pages_dev_ia_2023.pdf"]
  file_streams = [open(path, "rb") for path in file_paths]
  
  # Use the upload and poll SDK helper to upload the files, add them to the vector store,
  # and poll the status of the file batch for completion.
  file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files=file_streams
  )
  
  # You can print the status and the file counts of the batch to see the result of this operation.
  print(file_batch.model_dump_json(indent=2))
  print(file_batch.status)
  print(file_batch.file_counts)

VectorStore(id='vs_lrYKplXCKWpkkgtXUXzeanNt', created_at=1722355697, file_counts=FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1), last_active_at=1727676926, metadata={}, name='Certif 5 Pages IA 2023', object='vector_store', status='completed', usage_bytes=23160, expires_after=None, expires_at=None)
vector store already exists


In [36]:
# vs_lrYKplXCKWpkkgtXUXzeanNt (5 pages) ou vs_1Hgcfus8FuqgRINpJq7iAkfG (full)

assistant = client.beta.assistants.update(
  "asst_zgsWyBwKdWenyzpi9hGEl5h6",
  temperature=0.2,
  tool_resources={
  "file_search": {
    "vector_store_ids": ["vs_lrYKplXCKWpkkgtXUXzeanNt"]
  }
  }
)                   

In [ ]:
# asst_jbpMGP39cEMyhm6eD8JKO02r ou asst_zgsWyBwKdWenyzpi9hGEl5h6 (gpt-4o)

prompt = "Quelle est la composition exacte du jury de la certification?"
messages = test_assistant('asst_zgsWyBwKdWenyzpi9hGEl5h6', prompt)
print(messages)

Thread(id='thread_bDEbJR9ul2OmYUD7nIO0vbB6', created_at=1727340757, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))
Elapsed time: 0 minutes 0 seconds
Status: in_progress
Elapsed time: 0 minutes 2 seconds
Status: in_progress
Elapsed time: 0 minutes 5 seconds
Status: in_progress
Elapsed time: 0 minutes 7 seconds
Status: completed
Status: completed
Elapsed time: 0 minutes 10 seconds
{
  "status": "succeeded",
  "answer": "Le jury de la certification est composé de professionnels du secteur et de formateurs habilités. Ils sont responsables de l'évaluation des compétences des candidats à travers des épreuves certificatives, des cas pratiques, des mises en situation professionnelle, et des soutenances orales[0].",
  "citations": {
    "0": "reglement_specifique_5_pages_dev_ia_2023.pdf"
  }
}
['[0] reglement_specifique_5_pages_dev_ia_2023.pdf']
SyncCursorPage[Message](data=[Message(id='msg_poc2e5hyuVM439miEnFF8Az8', assistant_id='asst_zgsWy

In [87]:
# asst_jbpMGP39cEMyhm6eD8JKO02r ou asst_zgsWyBwKdWenyzpi9hGEl5h6 (gpt-4o)
for i in range(10):
  prompt = "Fais-moi un résumé du risque encouru en cas de fraude"
  messages = test_assistant('asst_zgsWyBwKdWenyzpi9hGEl5h6', prompt)
  print(messages)
  time.sleep(60)


Thread(id='thread_JOuaz3T5k0LDCbP64dZNlXcN', created_at=1722360551, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))
Elapsed time: 0 minutes 0 seconds
Status: in_progress
Elapsed time: 0 minutes 2 seconds
Status: in_progress
Elapsed time: 0 minutes 4 seconds
Status: in_progress
Elapsed time: 0 minutes 6 seconds
Status: completed
Status: completed
Elapsed time: 0 minutes 9 seconds
{
  "status": "failed",
  "answer": "Désolé, nous n'avons pas pu trouvé la réponse à votre question dans nos données",
  "citations": {}
}
SyncCursorPage[Message](data=[Message(id='msg_buzreg1RBiEcMXuhcilkIYeY', assistant_id='asst_zgsWyBwKdWenyzpi9hGEl5h6', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='```json\n{\n  "status": "failed",\n  "answer": "Désolé, nous n\'avons pas pu trouvé la réponse à votre question dans nos données",\n  "citations": {}\n}\n```'), type='text')], created_at=1722360557, incomplete_at=

In [59]:

prompt = "Who is Donald Trump?"
messages = test_assistant('asst_zgsWyBwKdWenyzpi9hGEl5h6', prompt)
print(messages)

Thread(id='thread_R7wyJzQaBKE4ecobRQH4ywvw', created_at=1722358114, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))
Elapsed time: 0 minutes 0 seconds
Status: queued
Elapsed time: 0 minutes 2 seconds
Status: in_progress
Elapsed time: 0 minutes 4 seconds
Status: completed
Status: completed
Elapsed time: 0 minutes 6 seconds
{
  "status": "failed",
  "answer": "Je suis désolé, je ne peux que fournir des news",
  "articles": []
}
SyncCursorPage[Message](data=[Message(id='msg_tjp8zTHoaz6zJoieXRtM7iV0', assistant_id='asst_zgsWyBwKdWenyzpi9hGEl5h6', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='```json\n{\n  "status": "failed",\n  "answer": "Je suis désolé, je ne peux que fournir des news",\n  "articles": []\n}\n```'), type='text')], created_at=1722358119, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_Wdpy4bfn8gQsrLqBgf6CMWL4', 

In [16]:

prompt = "Quels sont les éléments de preuves attendus pour le bloc de compétences 1? Donne 3 exemples"
messages = test_assistant('asst_zgsWyBwKdWenyzpi9hGEl5h6', prompt)
print(messages)

Thread(id='thread_2718sDQYDkW3zWzuRxmbc5SF', created_at=1727678464, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))
Elapsed time: 0 minutes 0 seconds
Status: in_progress
Elapsed time: 0 minutes 2 seconds
Status: in_progress
Elapsed time: 0 minutes 4 seconds
Status: in_progress
Elapsed time: 0 minutes 6 seconds
Status: in_progress
Elapsed time: 0 minutes 9 seconds
Status: completed
Status: completed
Elapsed time: 0 minutes 11 seconds
{
  "status": "succeeded",
  "answer": "Pour le bloc de compétences 1 de la certification Simplon, les éléments de preuves attendus incluent :

1. **Automatisation de l'extraction de données** : Automatiser l’extraction de données depuis un service web, une page web (scraping), un fichier de données, une base de données et un système big data en programmant le script adapté afin de pérenniser la collecte des données nécessaires au projet[0].

2. **Développement de requêtes SQL** : Développer des requêtes 